In [1]:
import pandas as pd

DATA_PATH = 'data/churn.xlsx'

df = pd.read_excel(DATA_PATH).drop(columns='customerID')
df.head(4)

,tenure,PhoneService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,34,Yes,One year,No,Mailed check,56.95,1889.5,No
2,2,Yes,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,45,No,One year,No,Bank transfer,42.30,1840.75,No


In [2]:
df.dtypes

tenure                int64
PhoneService         object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [3]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [4]:
df.dropna(inplace=True)
df.isnull().any()

tenure              False
PhoneService        False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges        False
Churn               False
dtype: bool

In [5]:
cat_vars = ['PhoneService', 'Contract', 'PaperlessBilling', 'PaymentMethod']
num_vars = ['tenure', 'MonthlyCharges', 'TotalCharges']
mod_vars = cat_vars + num_vars

In [6]:
X = df[mod_vars].copy()
y = pd.DataFrame(df['Churn'].copy())

In [7]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y['Churn'] = le.fit_transform(y['Churn'])

X  = pd.get_dummies(X)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

print(f'Shape X_train -> {X_train.shape}\nShape y_train -> {y_train.shape}\n\
Shape X_test  -> {X_test.shape}\nShape y_test  -> {y_test.shape}')

Shape X_train -> (5624, 14)
Shape y_train -> (5624, 1)
Shape X_test  -> (1407, 14)
Shape y_test  -> (1407, 1)


In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)

X_train = pd.DataFrame(scaler.transform(X_train), columns = X_train.columns)
X_test  = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

In [11]:
from sklearn.neural_network import MLPClassifier


clf = MLPClassifier(hidden_layer_sizes=(100, 50, 20), activation='logistic')
clf.fit(X_train, y_train)

c:\Users\jeanl\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 50, 20))

In [12]:
from sklearn.metrics import accuracy_score, f1_score

print(f'\
Accuracy -> {accuracy_score(y_test, clf.predict(X_test)) * 100 :.2f}%\n\
F1_Score -> {f1_score(y_test, clf.predict(X_test)) * 100 :.2f}%')

Accuracy -> 79.89%
F1_Score -> 53.38%
